# ........................................................................................................................................................

In [3]:
import pandas as pd
import numpy as np
import datetime
import warnings
import seaborn as sns
import prophet
import glob
import datetime as dt
warnings.filterwarnings("ignore")

### Part 0 - Data preparation

    - Original data (2021 Dec version)

In [4]:
data = pd.DataFrame()
for f in glob.glob("../IV Sets Clean data/*.xlsx"):
    df = pd.read_excel(f)
    data = data.append(df,ignore_index=True)
data = data[data['Cat.'].str.contains('Set')]
data=data.rename(columns={'FY Year':'Date','Units':'Quantity'})
data['Date']=pd.to_datetime(data['Date'])

ValueError: cannot assemble with duplicate keys

#### Select top 10 skus (high runners)

In [ ]:
sku_summary=data[data['Date']<"2021-12-01"].groupby(['SAP SKU'])['Quantity'].agg([('total_orders','sum')]).reset_index()
sku_summary['rank']=sku_summary['total_orders'].rank(method='dense',ascending=False)
sku_summary=sku_summary.sort_values(by=['total_orders'],ascending=False)
top_10 = list(sku_summary['SAP SKU'][sku_summary['rank']<=10])
sku_summary.head(10) # display top 10, high runners

### Part 1 - SKU level forecast

In [ ]:
#historical data at SKU level
historical_data=data[data['SAP SKU'].isin(top_10)].groupby(['SAP SKU','Date'])['Quantity'].agg([('Quantity','sum')]).reset_index()
sns.relplot(
    data=historical_data,
    x="Date", y="Quantity",col='SAP SKU',col_wrap=3,kind='line',facet_kws=dict(sharey=False)
)

#### We can see above the historical signal for each sku. We will highlight high peaks and drops and trends changes so that we can incorporate them into the prophet model to avoid overfitting.

In [ ]:
historical_data[['peak','drop']]=0
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='60593') & 
                          (historical_data['Date'].isin(["2020-03-01","2020-04-01","2020-10-01","2021-04-01"])) 
                        )]=1
historical_data['drop'][( (historical_data['SAP SKU']=='60593') & 
                          (historical_data['Date'].isin(["2020-05-01","2020-06-01","2020-07-01","2020-08-01",
                                            "2021-02-01","2021-07-01","2021-08-01","2021-09-01","2021-10-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='G30402M') & 
                          (historical_data['Date'].isin(["2018-04-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='273-004V') & 
                          (historical_data['Date'].isin(["2020-03-01","2021-05-01"])) 
                        )]=1
historical_data['drop'][( (historical_data['SAP SKU']=='273-004V') & 
                          (historical_data['Date'].isin(["2020-05-01","2021-02-01","2021-03-01",
                                            "2021-07-01","2021-08-01","2021-09-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='2420-0007') & 
                          (historical_data['Date'].isin(["2021-09-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='60693E') & 
                          (historical_data['Date'].isin(["2021-04-01"])) 
                        )]=1
historical_data['drop'][( (historical_data['SAP SKU']=='60693E') & 
                          (historical_data['Date'].isin(["2021-10-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='72504EB') & 
                          (historical_data['Date'].isin(["2019-05-01","2020-06-01","2021-04-01"])) 
                        )]=1
historical_data['drop'][( (historical_data['SAP SKU']=='72504EB') & 
                          (historical_data['Date'].isin(["2021-08-01","2021-09-01","2021-10-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='G30302M') & 
                          (historical_data['Date'].isin(["2021-03-01","2021-04-01"])) 
                        )]=1
historical_data['drop'][( (historical_data['SAP SKU']=='G30302M') & 
                          (historical_data['Date'].isin(["2021-06-01","2021-07-01","2021-08-01","2021-09-01","2021-10-01","2021-11-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='72304B') & 
                          (historical_data['Date'].isin(["2019-01-01","2020-04-01","2020-12-01","2021-06-01","2021-07-01","2021-11-01"])) 
                        )]=1
historical_data['drop'][( (historical_data['SAP SKU']=='72304B') & 
                          (historical_data['Date'].isin(["2020-05-01","2020-08-01","2020-09-01","2021-03-01","2021-04-01","2021-09-01","2021-10-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='60093E') & 
                          (historical_data['Date']<="2017-09-01") &
                          (historical_data['Date']>="2016-10-01") 
                        )]=1
historical_data['drop'][( (historical_data['SAP SKU']=='60093E') & 
                          (historical_data['Date'].isin(["2021-03-01","2021-09-01","2021-10-01"])) 
                        )]=1
#......................................................................................................................

#### use prophet model with special events as regressors.

In [ ]:
def generate_stats(material,snapshot_date=pd.to_datetime("2021-12-01"),h=24):
    
    original_data=historical_data.copy()
    original_data = original_data[((original_data['SAP SKU']==material) &
                                    (original_data['Date']<snapshot_date))]
    
    if material=="72304B":
      original_data = original_data[original_data['Date']>="2017-03-01"]
    elif material=="72504EB":
      original_data = original_data[original_data['Date']>="2016-03-01"]
    
    #whether to fit auto.seasonality (if series not intermittent)
    zeros=np.sum(original_data.Quantity==0)/(original_data.Quantity.shape[0])
    if zeros<=0.4:
      yearly=False
    else:
      yearly='auto'
    
    #define regressor functions for the prophet model
    def peak(ds):
      dates=pd.to_datetime(ds)
      return original_data['peak'][original_data['Date'].isin(dates)] 
    def drop(ds):
      dates=pd.to_datetime(ds)
      return original_data['drop'][original_data['Date'].isin(dates)] 
    

    input_data= original_data.copy()
    input_data=input_data[['Date','Quantity']].rename(columns={'Date':'ds','Quantity':'y'})
    
    #fitting the different models one by one
    input_data['drop']=drop(input_data['ds'])
    input_data['peak']=peak(input_data['ds'])
    model=prophet.Prophet(yearly_seasonality=yearly,weekly_seasonality = False,daily_seasonality = False)
    model.add_regressor('drop')
    model.add_regressor('peak')
    model.fit(input_data)
    forecast_timeline=model.make_future_dataframe(periods = h,freq="MS")
    forecast_timeline[['drop','peak']]=0
    prophet_forecast=model.predict(forecast_timeline)

    #consolidating the different forecasts
    all_fcast_1 = prophet_forecast[['ds','yhat','yhat_lower','yhat_upper']][prophet_forecast['ds']>=snapshot_date]
    all_fcast_1=all_fcast_1.rename(columns={"ds":"Date","yhat":"prophet_fcast","yhat_lower":"prophet_lower","yhat_upper":"prophet_upper"})
    all_fcast_1['prophet_fcast'][all_fcast_1['prophet_fcast']<0]=0
    all_fcast_1['prophet_lower'][all_fcast_1['prophet_lower']<0]=0
    all_fcast_1['prophet_upper'][all_fcast_1['prophet_upper']<0]=0
    all_fcast_1['series_type']="regular"
    all_fcast_1['input_category']="original_with_regressors"
    all_fcast_1['SAP.SKU']=material
    all_fcast_1['Snapshot']=pd.to_datetime(snapshot_date)
      
    return all_fcast_1
    

In [ ]:
#material=top_10[1]
#generate_stats(material)

In [ ]:
this_snapshot= dt.datetime.today().replace(minute=0, hour=0, second=0, microsecond=0,day=1)
all_stats=[]
for material in top_10:
    all_stats=all_stats+[generate_stats(material=material,snapshot_date=pd.to_datetime(this_snapshot))]
all_stats=pd.concat(all_stats,ignore_index=True)
all_stats.info()

In [ ]:
all_stats

#### Export the results for the sku level forecast

In [ ]:
#historical_data.to_excel("./stats_output/sku_level/historical_data_"+this_snapshot+".xlsx",index=False)
all_stats.to_excel("../Emea_high_runner_SKUlvl.xlsx",index=False)

#### Generate stats for the past few snapshots

In [ ]:
#snapshots = pd.date_range(start="2021-07-01",end="2021-11-01",freq='MS')

In [ ]:
#(.........might take time to run: can be improved with apply/high perf computing etc .............)
#all_stats=[]
#for snapshot in snapshots:
#    for material in top_10:
#        all_stats=all_stats+[generate_stats(material=material,snapshot_date=snapshot)]
#all_stats=pd.concat(all_stats,ignore_index=True)
#all_stats.info()

In [ ]:
#all_stats.to_excel("./stats_output/sku_level/generated_forecast_previous_snapshots.xlsx",index=False)

### Part 2 - Sku & Cluster level forecast.

#### prepare the input data by identifying start date and end date to focus on relevant signals to simulate

In [ ]:
historical_data=data[ (data['SAP SKU'].isin(top_10)) & (data['Cluster'].notnull())]
historical_data=historical_data.groupby(['SAP SKU','Cluster','Date'])['Quantity'].agg([('Quantity','sum')]).reset_index()

max_date=max(historical_data['Date']) - pd.DateOffset(months=18) # to filter out series with no demand for the last 18 months

temp = historical_data[historical_data['Quantity']!=0].groupby(['SAP SKU','Cluster'])['Date'].agg([('start_date','min'),('end_date','max')]).reset_index()

historical_data = historical_data.merge(temp,how='left')
historical_data = historical_data[historical_data['start_date'].notnull()] #remove null signals

historical_data = historical_data[historical_data['Date']>=historical_data['start_date']] # re-frame each time series
historical_data = historical_data[historical_data['end_date']>=max_date] # consider we no longer have demand if no demand since the last 18 months

historical_data.info()

#### add peaks and drops noticed at sku level

In [ ]:
historical_data[['peak','drop']]=0
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='60593') & 
                          (historical_data['Date'].isin(["2020-03-01","2020-04-01","2020-10-01","2021-04-01"])) 
                        )]=1
historical_data['drop'][( (historical_data['SAP SKU']=='60593') & 
                          (historical_data['Date'].isin(["2020-05-01","2020-06-01","2020-07-01","2020-08-01",
                                            "2021-02-01","2021-07-01","2021-08-01","2021-09-01","2021-10-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='G30402M') & 
                          (historical_data['Date'].isin(["2018-04-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='273-004V') & 
                          (historical_data['Date'].isin(["2020-03-01","2021-05-01"])) 
                        )]=1
historical_data['drop'][( (historical_data['SAP SKU']=='273-004V') & 
                          (historical_data['Date'].isin(["2020-05-01","2021-02-01","2021-03-01",
                                            "2021-07-01","2021-08-01","2021-09-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='2420-0007') & 
                          (historical_data['Date'].isin(["2021-09-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='60693E') & 
                          (historical_data['Date'].isin(["2021-04-01"])) 
                        )]=1
historical_data['drop'][( (historical_data['SAP SKU']=='60693E') & 
                          (historical_data['Date'].isin(["2021-10-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='72504EB') & 
                          (historical_data['Date'].isin(["2019-05-01","2020-06-01","2021-04-01"])) 
                        )]=1
historical_data['drop'][( (historical_data['SAP SKU']=='72504EB') & 
                          (historical_data['Date'].isin(["2021-08-01","2021-09-01","2021-10-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='G30302M') & 
                          (historical_data['Date'].isin(["2021-03-01","2021-04-01"])) 
                        )]=1
historical_data['drop'][( (historical_data['SAP SKU']=='G30302M') & 
                          (historical_data['Date'].isin(["2021-06-01","2021-07-01","2021-08-01","2021-09-01","2021-10-01","2021-11-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='72304B') & 
                          (historical_data['Date'].isin(["2019-01-01","2020-04-01","2020-12-01","2021-06-01","2021-07-01","2021-11-01"])) 
                        )]=1
historical_data['drop'][( (historical_data['SAP SKU']=='72304B') & 
                          (historical_data['Date'].isin(["2020-05-01","2020-08-01","2020-09-01","2021-03-01","2021-04-01","2021-09-01","2021-10-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['SAP SKU']=='60093E') & 
                          (historical_data['Date']<="2017-09-01") &
                          (historical_data['Date']>="2016-10-01") 
                        )]=1
historical_data['drop'][( (historical_data['SAP SKU']=='60093E') & 
                          (historical_data['Date'].isin(["2021-03-01","2021-09-01","2021-10-01"])) 
                        )]=1
#......................................................................................................................

#### add peaks and drops noticed at cluster level

In [ ]:
#......................................................................................................................
historical_data['peak'][( (historical_data['Cluster']=='Arab Peninsula') & 
                          (historical_data['Date'].isin(["2021-08-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['Cluster']=='BNL') & 
                          (historical_data['Date'].isin(["2017-08-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['Cluster']=='IB') & 
                          (historical_data['Date'].isin(["2020-03-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['Cluster']=='CEE') & 
                          (historical_data['Date'].isin(["2020-11-01"])) 
                        )]=1
#......................................................................................................................
historical_data['drop'][( (historical_data['Cluster']=='FR') & 
                          (historical_data['Date']>="2020-04-01") &
                          (historical_data['SAP SKU']=="273-004V")
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['Cluster']=='GSA') & 
                          (historical_data['Date'].isin(["2020-03-01","2021-09-01"])) 
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['Cluster']=='ISR') & 
                          (historical_data['Date'].isin(["2017-11-01"])) 
                        )]=1
historical_data['drop'][( (historical_data['Cluster']=='ISR') & 
                          (historical_data['Date']<="2018-08-01") &
                          (historical_data['Date']>="2018-01-01")
                        )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['Cluster']=='IT') & 
                          (historical_data['Date'].isin(["2020-03-01","2021-04-01"])) 
                        )]=1
historical_data['drop'][( (historical_data['Cluster']=='IT') & 
                          (historical_data['Date'].isin(["2021-08-01","2021-09-01","2021-10-01"]))
                         )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['Cluster']=='NOR') & 
                          (historical_data['Date'].isin(["2021-11-01"])) 
                        )]=1
historical_data['drop'][( (historical_data['Cluster']=='NOR') & 
                          (historical_data['Date'].isin(["2021-08-01","2021-09-01"]))
                         )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['Cluster']=='Saudi Arabia') & 
                          (historical_data['Date'].isin(["2018-04-01"])) 
                        )]=1
historical_data['drop'][( (historical_data['Cluster']=='Saudi Arabia') & 
                          (historical_data['Date']<="2019-03-01") &
                          (historical_data['Date']>="2018-05-01")
                         )]=1
#......................................................................................................................
historical_data['peak'][( (historical_data['Cluster']=='UK') & 
                          (historical_data['Date'].isin(["2020-10-01","2020-11-01"])) 
                        )]=1
historical_data['peak'][( (historical_data['Cluster']=='UK') & 
                          (historical_data['Date'].isin(["2020-04-01","2021-04-01"])) 
                        )]=2
historical_data['drop'][( (historical_data['Cluster']=='UK') & 
                          (historical_data['Date'].isin(["2020-08-01","2021-02-01"]))
                         )]=1
historical_data['drop'][( (historical_data['Cluster']=='UK') & 
                          (historical_data['Date'].isin(["2021-07-01","2021-08-01","2021-09-01","2021-10-01"]))
                         )]=2

In [ ]:
def generate_stats2(material,cluster,snapshot_date=pd.to_datetime("2021-12-01"),h=24):
    
    original_data=historical_data.copy()
    original_data = original_data[((original_data['SAP SKU']==material) &
                                   (original_data['Cluster']==cluster) &
                                    (original_data['Date']<snapshot_date))]
    
    if material=="72304B":
      original_data = original_data[original_data['Date']>="2017-03-01"]
    elif material=="72504EB":
      original_data = original_data[original_data['Date']>="2016-03-01"]
    
    #whether to fit auto.seasonality (if series not intermittent)
    zeros=np.sum(original_data.Quantity==0)/(original_data.Quantity.shape[0])
    if zeros<=0.4:
      yearly=False
    else:
      yearly='auto'
    
    #define regressor functions for the prophet model
    def peak(ds):
      dates=pd.to_datetime(ds)
      return original_data['peak'][original_data['Date'].isin(dates)] 
    def drop(ds):
      dates=pd.to_datetime(ds)
      return original_data['drop'][original_data['Date'].isin(dates)] 
    

    input_data= original_data.copy()
    input_data=input_data[['Date','Quantity']].rename(columns={'Date':'ds','Quantity':'y'})
    
    input_data['drop']=drop(input_data['ds'])
    input_data['peak']=peak(input_data['ds'])
    model=prophet.Prophet(yearly_seasonality=yearly,weekly_seasonality = False,daily_seasonality = False)
    model.add_regressor('drop')
    model.add_regressor('peak')
    model.fit(input_data)
    forecast_timeline=model.make_future_dataframe(periods = h,freq="MS")
    forecast_timeline[['drop','peak']]=0
    prophet_forecast=model.predict(forecast_timeline)

    #consolidating the different forecasts
    all_fcast_1 = prophet_forecast[['ds','yhat','yhat_lower','yhat_upper']][prophet_forecast['ds']>=snapshot_date]
    all_fcast_1=all_fcast_1.rename(columns={"ds":"Date","yhat":"prophet_fcast","yhat_lower":"prophet_lower","yhat_upper":"prophet_upper"})
    all_fcast_1['prophet_fcast'][all_fcast_1['prophet_fcast']<0]=0
    all_fcast_1['prophet_lower'][all_fcast_1['prophet_lower']<0]=0
    all_fcast_1['prophet_upper'][all_fcast_1['prophet_upper']<0]=0
    all_fcast_1['series_type']="regular"
    all_fcast_1['input_category']="original_with_regressors"
    all_fcast_1['SAP.SKU']=material
    all_fcast_1['Cluster']=cluster
    all_fcast_1['Snapshot']=pd.to_datetime(snapshot_date)
      
    return all_fcast_1
    

In [ ]:
#generate_stats2(material="60593",cluster="UK",snapshot_date=pd.to_datetime('2021-07-01'))

In [ ]:
#(... might take time to run: can be improved with apply/high perf computing etc ...)
this_snapshot= dt.datetime.today().replace(minute=0, hour=0, second=0, microsecond=0,day=1)
all_stats=[]
for material in top_10:
    for cluster in historical_data['Cluster'][historical_data['SAP SKU']==material].unique():
        all_stats=all_stats+[generate_stats2(material=material,cluster=cluster,snapshot_date=pd.to_datetime(this_snapshot))]
all_stats=pd.concat(all_stats,ignore_index=True)
all_stats.info()

In [ ]:
all_stats

In [ ]:
#historical_data.to_excel("./stats_output/sku_x_cluster_level//historical_data_SKUxCluster_"+this_snapshot+".xlsx",index=False)
all_stats.to_excel("../Highrunner_Forecast/generated_forecast_SKUxCluster_"+this_snapshot.strftime("%m-%d-%Y")+".xlsx",index=False)

#### Generate stats for the past few snapshots

In [ ]:
#snapshots = pd.date_range(start="2021-07-01",end="2021-11-01",freq='MS')

In [ ]:
#(....might take time to run: can be improved with apply/high perf computing etc .......)
#all_stats=[]
#for snapshot in snapshots:
#    for material in top_10:
#        for cluster in historical_data['Cluster'][historical_data['SAP SKU']==material].unique():
#            all_stats=all_stats+[generate_stats2(material=material,cluster=cluster,snapshot_date=snapshot)]
#all_stats=pd.concat(all_stats,ignore_index=True)
#all_stats.info()

In [ ]:
#all_stats.to_excel("./stats_output/generated_forecast_SKUxCluster_previous_snapshots.xlsx",index=False)

# .....................................................................................................................................................................